# Chemical equilibrium with fixed fugacity

This tutorial demonstrates the calculation of calcite (CaCO{{_3}}) solubility in a saline
aqueous solution for given fugacity of CO{{_2}}. 

The code block below produces a chemical system containing an aqueous phase (to
model our saline aqueous solution) and a solid mineral phase (to model our
calcite mineral). 

In [15]:
import reaktoro as rkt

db = rkt.PhreeqcDatabase("phreeqc.dat")

solution = rkt.AqueousPhase(rkt.speciate("H O Na Cl C Ca"))
solution.setActivityModel(rkt.ActivityModelHKF())

calcite = rkt.MineralPhase("Calcite")

system = rkt.ChemicalSystem(db, solution, calcite)

In this chemical equilibrium calculation, the following properties are
constrained:

* temperature;
* pressure; and
* fugacity of CO{{_2}};

Because of the fugacity constraint, the chemical equilibrium calculation we
will perform next presumes that the system is open to CO{{_2}}. During the
calculation, enough CO{{_2}} is allowed to leave or enter the system so that
the fugacity constraint is satisfied.

The next step is to create an object of class `EquilibriumSpecs` containing the
specifications for the chemical equilibrium calculation we want to perform.

In [16]:
specs = rkt.EquilibriumSpecs(system)
specs.temperature()
specs.pressure()
specs.fugacity("CO2")

Once we have created this specifications object, we can now create our
equilibrium solver:

In [17]:
solver = rkt.EquilibriumSolver(specs)

Let's now create an initial chemical state for our chemical system, one that represents a 1 molal NaCl aqueous solution mixed with 10 g of calcite mineral:

In [18]:
state = rkt.ChemicalState(system)
state.temperature(50.0, "celsius")
state.pressure(10.0, "bar")
state.set("H2O", 1.0, "kg")
state.set("Na+", 1.0, "mol")
state.set("Cl-", 1.0, "mol")
state.set("Calcite", 10, "g")

Note that this state is not yet in chemical equilibrium! The `state` object will be equilibrated later.

Let's now define the temperature, pressure and fugacity conditions we want to
impose at equilibrium for this aqueous solution. We do this by using an object
of class `EquilibriumConditions`:

In [19]:
conditions = rkt.EquilibriumConditions(specs)
conditions.temperature(50.0, "celsius")
conditions.pressure(10.0, "bar")
conditions.fugacity("CO2", 1.0, "bar")

We have everything we need now to perform the equilibrium calculation:

* an equilibrium solver (the `solver` object of class `EquilibriumSolver`);
* a chemical state to equilibrate (the `state` object of class `ChemicalState`); and
* the conditions we are imposing for this equilibrium state (the `conditions`
  object of class `EquilibriumConditions`).

The code below will perform the equilibrium calculation and when it is
finished, our `state` object will be in a chemical equilibrium state:

In [20]:
result = solver.solve(state, conditions)

It's always advisable to verify if the calculation succeeded:

In [21]:
print("Successful computation!" if result.optima.succeeded else "Computation has failed!")

Successful computation!


Let's check the computed chemical equilibrium state: 

In [22]:
print(state)

+-----------------+-------------+------+
| Property        |       Value | Unit |
+-----------------+-------------+------+
| Temperature     |      323.15 |    K |
| Pressure        |       1e+06 |   Pa |
| Element Amount: |             |      |
| :: H            |     111.012 |  mol |
| :: C            |    0.127204 |  mol |
| :: O            |     55.8605 |  mol |
| :: Na           |           1 |  mol |
| :: Cl           |           1 |  mol |
| :: Ca           |    0.099909 |  mol |
| Species Amount: |             |      |
| :: CO3-2        | 7.48007e-06 |  mol |
| :: H+           | 1.15634e-06 |  mol |
| :: H2O          |     55.4941 |  mol |
| :: CO2          |   0.0151464 |  mol |
| :: (CO2)2       |       1e-16 |  mol |
| :: HCO3-        |   0.0203079 |  mol |
| :: CH4          |       1e-16 |  mol |
| :: Ca+2         |   0.0116215 |  mol |
| :: CaCO3        | 5.67868e-06 |  mol |
| :: CaHCO3+      | 0.000604289 |  mol |
| :: CaOH+        | 6.19407e-10 |  mol |
| :: Cl-        

Well, there is no pH in this table. This is because `ChemicalState` objects
only store temperature, pressure, and species amounts. 

What we need now is an object of class `AqueousProps`. This class was designed
for a more convenient inspection of thermodynamic and chemical properties
related to aqueous solutions. Let's create an `AqueousProps` object and print
it:

In [23]:
aprops = rkt.AqueousProps(state)
print(aprops)

+--------------------------+-------------+-------+
| Property                 |       Value |  Unit |
+--------------------------+-------------+-------+
| Temperature              |      323.15 |     K |
| Pressure                 |       1e+06 |    Pa |
| Ionic Strength (Effect.) |     1.03229 | molal |
| Ionic Strength (Stoich.) |      1.0357 | molal |
| pH                       |      6.1347 |       |
| pE                       |     1.71539 |       |
| Eh                       |    0.109991 |     V |
| Element Molality:        |             |       |
| :: C                     |   0.0395363 | molal |
| :: Na                    |     1.00026 | molal |
| :: Cl                    |     1.00026 | molal |
| :: Ca                    |   0.0122346 | molal |
| Species Molality:        |             |       |
| :: CO3-2                 | 7.48201e-06 | molal |
| :: H+                    | 1.15664e-06 | molal |
| :: CO2                   |   0.0151504 | molal |
| :: (CO2)2                | 1.

Great! We can now see that pH is exactly 2, as we enforced, and temperature and
pressure are 50 °C and 10 bar (but displayed in the table as 323.15 K and
10<sup>6</sup> Pa respectively).

Let's now compare the amounts of elements at equilibrium state with that at the
initial state. 

While both elements C and O are numerically conserved (i.e., their final and
initial amounts are equal up to machine precision, which is approximately
1.11e-16), the same cannot be said about element H.

This is because H should not be expected to be preserved in this particular
equilibrium problem, because our chemical system is implicitly open to
H<sup>+</sup>. It is not thermodynamically possible in general to impose,
simultaneously, temperature, pressure, pH and conservation of all chemical
elements and electric charge. 

To impose pH, we need to discard either temperature or pressure constraints, or
make the system open to a substance. Because pH is not strongly sensitive to
temperature and pressure changes, equilibrium calculations with fixed pH are
usually done assuming that an unknown amount of a specified titrant (acid or
base) is added into (or removed from) the system.

```{note}
The current implicit behavior in Reaktoro in which pH constraint presumes that
the system is open to H<sup>+</sup> will be removed in the near future. We will
instead have a more explicit approach in which pH constraints are specified
together with a titrant for which the system is open to. 

Next time you check this tutorial, you may find that `specs.pH("H+")` is needed
instead of `specs.pH()` to achieve identical results we have now. 

This will more clearly specify that the system is open to H<sup>+</sup>. With
this planned change, we'll even be able to specify other titrants, such as HCl,
NaCl, CO2, etc., using the syntax `specs.pH("HCl")`.
```

With this we conclude this tutorial. Stay tuned for an upcoming revision!